In [ ]:
from __future__ import print_function

### Data generator

For selected font, script will generate each of above character (a-z, A-Z, 0-9) at nine different position to move text by one pixel in right-left and top-bottom direction. 

#### NOTE: If you want generate some new data use kernel == python2, because ttfquery can cause problems at python3
Also you can use generated images from Synthetic_dataset

#### Imports

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import ttfquery.findsystem 
import string
import ntpath
import numpy as np
import os
import glob

### Generator

Input params:

    fontSize
    imgSize
    position
    font_list - list of fonts which will be used

### Wordbag creating and exporting to .txt file

In [ ]:
def words_parser(inp_str):
    lower_case_list = list(string.ascii_lowercase)
    words = []
    i = 0
    j = 0
    flag = False
    while i < len(inp_str):
        while inp_str[i] in string.ascii_letters:
            i+=1  
            flag = True
        if flag and i>j+3: # take words with len > 3
            words.append(inp_str.lower()[j:i])
        i += 1
        j = i
        flag = False
    return words

def words_generator(path='words.txt'):
    
    try:
        from sklearn.datasets import fetch_20newsgroups
        
        wordbag = fetch_20newsgroups(subset='all', categories=['sci.space'], shuffle=True, random_state=42)
        words = []; i = 0
        for text in wordbag.data[:100][:]:
            tmp = words_parser(text)
            words = np.hstack([words,tmp])
        words = words.tolist()
        print(len(words), 'words were generated')
        f = open('words.txt','w')
        for word in words:
            f.write(word + '\n')
        f.close()
    
    except:
        print('Install sklearn to generate text!')

In [ ]:
words_generator()

In [ ]:
def words_bag_reader(filename):
    #reading wordbag
    words_bag = []
    try:
        with open(filename, 'r') as f:
            words_bag.append(f.read())
            words_bag = list(words_bag)
        words_bag = words_bag[0].split('\n')
    except:
        print("File with words doesnt exist!!!")
        
    return words_bag

def data_generator(fonts, directory='Synthetic_dataset', img_per_class=20, img_size=(128,128), fontSize=20):
    """
    fonst -- list of fonts which will be classaficated
    words randomly can be in lower case, upper case and only first char is upper
    words are taken from wordbag file
    """
        
    words_bag = words_bag_reader('words.txt')
    
    #creating directory
    dataset_path = os.path.join (os.getcwd(), directory)
    if not os.path.exists(dataset_path):
        os.makedirs(dataset_path) 
    
    #system fonts
    all_fonts = ttfquery.findsystem.findFonts()
    #creating paths to fonts
    cases = [string.lower, string.upper, string.capitalize]
    fonts.sort()
    
    for cur_font in fonts:
        for i in range(img_per_class):
            for sys_font in all_fonts:
                sys_lower = sys_font.lower()
                cur_lower = cur_font.lower()
                if cur_lower+'.' in sys_lower:# dot is used to get regular style
                    path = sys_font
                    font = ImageFont.truetype(path, fontSize)
                    text = np.random.choice(words_bag)
                    text = np.random.choice(cases)(text)
                    text_width, text_height = font.getsize(text)
                    flag = (text_width >= img_size[0] or text_height >= img_size[1])
                    while flag:
                        text = np.random.choice(words_bag)
                        text = np.random.choice(cases)(text)
                        text_width, text_height = font.getsize(text)
                        flag = (text_width >= img_size[0] or text_height >= img_size[1])
                    image = Image.new("RGB", (text_width, text_height), (255,255,255))
                    draw = ImageDraw.Draw(image)
                    draw.text((0,0), text, (0,0,0), font=font)
                    file_name = str(cur_lower) + '_' + str(i)+'.jpg'
                    file_name = os.path.join(dataset_path,file_name)
                    image.save(file_name)
                    break   
                    
def data_generator_folders(fonts, directory='Synthetic_dataset', img_per_class=20, img_size=(128,128), fontSize=20):
    """
    fonst -- list of fonts which will be classaficated
    words randomly can be in lower case, upper case and only first char is upper
    words are taken from wordbag file
    """
    
    words_bag = words_bag_reader('words.txt')
        
    #creating directory
    dataset_path = os.path.join (os.getcwd(), directory)
    if not os.path.exists(dataset_path):
        os.makedirs(dataset_path) 
    
    #system fonts
    all_fonts = ttfquery.findsystem.findFonts()
    #creating paths to fonts
    cases = [string.lower, string.upper, string.capitalize]
    fonts.sort()
    for cur_font in fonts:
        for i in range(img_per_class):
            for sys_font in all_fonts:
                sys_lower = sys_font.lower()
                cur_lower = cur_font.lower()
                if cur_lower+'.' in sys_lower:# dot is used to get regular style
                    path = sys_font
                    font = ImageFont.truetype(path, fontSize)
                    text = np.random.choice(words_bag)
                    text = np.random.choice(cases)(text)
                    text_width, text_height = font.getsize(text)
                    flag = (text_width >= img_size[0] or text_height >= img_size[1])
                    while flag:
                        text = np.random.choice(words_bag)
                        text = np.random.choice(cases)(text)
                        text_width, text_height = font.getsize(text)
                        flag = (text_width >= img_size[0] or text_height >= img_size[1])
                    image = Image.new("RGB", (text_width, text_height), (255,255,255))
                    draw = ImageDraw.Draw(image)
                    draw.text((0,0), text, (0,0,0), font=font)
                    file_name = str(i)+'.jpg'
                    font_dir = os.path.join(dataset_path, cur_lower)
                    if not os.path.exists(font_dir):
                        os.makedirs(font_dir)
                    file_name = os.path.join(font_dir,file_name)
                    image.save(file_name)
                    break   
                    
                    
                    
def data_generator_folders_train_test(fonts, words_bag, directory='Synthetic_dataset', img_per_class=20, \
                                      img_size=(128,128), fontSize=18):
    """
    fonst -- list of fonts which will be classaficated
    words randomly can be in lower case, upper case and only first char is upper
    words are taken from wordbag file
    """
    #creating directory
    dataset_path = os.path.join (os.getcwd(), directory)
    if not os.path.exists(dataset_path):
        os.makedirs(dataset_path) 
    
    #system fonts
    all_fonts = ttfquery.findsystem.findFonts()
    #creating paths to fonts
    cases = [string.lower, string.upper, string.capitalize]
    fonts.sort()
    for cur_font in fonts:
        for i in range(img_per_class):
            for sys_font in all_fonts:
                sys_lower = sys_font.lower()
                cur_lower = cur_font.lower()
                if cur_lower+'.' in sys_lower:# dot is used to get regular style
                    path = sys_font
                    font = ImageFont.truetype(path, fontSize)
                    text = np.random.choice(words_bag)
                    text = np.random.choice(cases)(text)
                    text_width, text_height = font.getsize(text)
                    flag = (text_width >= img_size[0] or text_height >= img_size[1])
                    while flag:
                        text = np.random.choice(words_bag)
                        text = np.random.choice(cases)(text)
                        text_width, text_height = font.getsize(text)
                        flag = (text_width >= img_size[0] or text_height >= img_size[1])
                    image = Image.new("RGB", (text_width, text_height), (255,255,255))
#                     image = Image.new("RGB", img_size, (255,255,255))
                    draw = ImageDraw.Draw(image)
                    draw.text((0,0), text, (0,0,0), font=font)
                    file_name = str(i)+'.jpg'
                    font_dir = os.path.join(dataset_path, cur_lower)
                    if not os.path.exists(font_dir):
                        os.makedirs(font_dir)
                    file_name = os.path.join(font_dir,file_name)
                    image.save(file_name)
                    break   

In [ ]:
#for ubuntu
fonts_list = ['Arial', 'Verdana', 'Comic_Sans_MS', 'Courier_New', 'Times_New_Roman', 'Impact', 'Georgia', 'Trebuc', \
             'Andalemo', 'Lato-Regular']
#for windows
fonts_list = ['Arial', 'Verdana', 'Comic', 'Cour', 'Times', 'Impact', 'Georgia', 'Trebuc', \
             'Alfredo', 'Borealis']
data_generator_folders(fonts=fonts_list, directory='Synthetic_dataset_words')

In [ ]:
#for ubuntu
fonts_list = ['Arial', 'Verdana', 'Comic_Sans_MS', 'Courier_New', 'Times_New_Roman', 'Impact', 'Georgia', 'Trebuc', \
             'Andalemo', 'Lato-Regular']
#for windows
fonts_list = ['Arial', 'Verdana', 'Comic', 'Cour', 'Times', 'Impact', 'Georgia', 'Trebuc', \
             'Corbel', 'Alger']
img_per_class=100
N = len(fonts_list)*img_per_class
words_bag = words_bag_reader('words.txt')
words_bag_set = np.random.choice(words_bag, size=3*N, replace=False)
words_bag_train = words_bag_set[:N]
words_bag_val = words_bag_set[N:2*N]
words_bag_test = words_bag_set[2*N:]
data_generator_folders_train_test(fonts_list, words_bag_train, 'Synthetic_dataset/train/', img_per_class)
data_generator_folders_train_test(fonts_list, words_bag_val, 'Synthetic_dataset/val/', img_per_class)
data_generator_folders_train_test(fonts_list, words_bag_test, 'Synthetic_dataset/test/', img_per_class)

### Data preparation

Here python3 is recommended

#### Imports

In [ ]:
import numpy as np
import keras
import tensorflow as tf
import os
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization 
from keras.layers import Conv2D, ActivityRegularization, MaxPooling2D
from keras.backend import resize_images, reshape
from keras.optimizers import SGD, Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 4} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

### ImageDataGenerator usage
We will use it

### Datagen for train, validation and test

For training we use train and validation data.
After training, test score is calculated by cross validation at test data

In [ ]:
def datagen(model, directory_train, directory_val, directory_test, batch_size=16, epochs=10, verbose=1,\
            nb_train_samples = 2000, nb_validation_samples = 800, nb_test_samples = 1000, img_size=(128,128)):
    """
    creating train/validaton generators for model fitting
    """
    # this is the augmentation configuration we will use for training
    train_datagen = ImageDataGenerator(rescale = 1./255.)

    # this is the augmentation configuration we will use for testing:
    # only rescaling
    test_datagen = ImageDataGenerator(rescale = 1./255.,)

    train_generator = train_datagen.flow_from_directory(
        directory=directory_train,
        batch_size=batch_size,   
        color_mode='grayscale',
        #save_to_dir='synt_aug/',
        target_size=img_size)
    
    validation_generator = test_datagen.flow_from_directory(
        directory=directory_val,
        batch_size=batch_size,
        color_mode='grayscale',
        target_size=img_size)
    
    test_generator = test_datagen.flow_from_directory(
        directory=directory_test,
        batch_size=batch_size,
        color_mode='grayscale',
        target_size=img_size)
 
    history = model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size,
        verbose=verbose,
        workers=1,#f you want multiprocessing change it
        use_multiprocessing=False,)
    
    score, acc = model.evaluate_generator(generator=test_generator, steps=nb_test_samples, verbose = 1)
    print('Score is', score, acc)
    
    return history, [score, acc]

## Choose architecture
#### First iteration

Architectures:

    1) CONV - POOL - DENSE - OUTPUT
    2) CONV - CONV - POOL - DENSE - OUTPUT
    3) CONV - CONV - CONV - POOL - DENSE - OUTPUT
Here we dont use any regularizers, normalizations

In [ ]:
input_shape = (32,32,1)
models = []
models.append(Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D(pool_size=(2, 2),strides=2),
    Flatten(),
    Dense(32, activation='relu'),
    Dense(10, activation='softmax')
]))

models.append(Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D(pool_size=(2, 2),strides=2),
    Flatten(),
    Dense(32, activation='relu'),
    Dense(10, activation='softmax')
]))

models.append(Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D(pool_size=(2, 2),strides=2),
    Flatten(),
    Dense(32, activation='relu'),
    Dense(10, activation='softmax')
]))

In [ ]:
scorelist = []
for model in models:
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    history, score = datagen(model,'./Synthetic_dataset/train', './Synthetic_dataset/val', './Synthetic_dataset/test',\
                      verbose=0, img_size=(32,32), batch_size=32, epochs=30, nb_train_samples=1000,\
                      nb_validation_samples=200, nb_test_samples = 200)
    scorelist.append(score[1])
print(scorelist)
argmax = np.argmax(scorelist)
print('The best model is model ', argmax+1)
model = models[argmax]

### Tuning process

In this section we will use BatchNormalization and Dropout.
Parameters will be tuning:

    dropout_1 -- uniform in [0, 1]
    dropout_2 -- uniform in [0, 1]
    second_conv -- adds conv - pool before dense
    second_dense -- adds one more dense 
    dense_units -- choice in [32, 64, 128]
    
Batch size equals 32, it is good choice for this task.

For net tunning we will take model 3.

We use hyperas for hyperparameters tunning.
There are some preparations for hyperas usage.

In [ ]:
def data():
    img_size = (32, 32)
    train_datagen = ImageDataGenerator(rescale = 1./255.)
    test_datagen = ImageDataGenerator(rescale = 1./255.,)

    train_generator = train_datagen.flow_from_directory(
        directory='./Synthetic_dataset/train',
        batch_size=32,   
        color_mode='grayscale',
        target_size=img_size)
    
    validation_generator = test_datagen.flow_from_directory(
        directory='./Synthetic_dataset/val',
        batch_size=32,
        color_mode='grayscale',
        target_size=img_size)
    
    test_generator = test_datagen.flow_from_directory(
        directory='./Synthetic_dataset/test',
        batch_size=32,
        color_mode='grayscale',
        target_size=img_size)
    
    return train_generator, validation_generator, test_generator

def model(train_generator, validation_generator, test_generator):
    
    nb_train_samples = 1000
    nb_validation_samples = 200
    nb_test_samples = 200
    epochs = 30
    batch_size = 32
    
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 1)))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 1)))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2),strides=2))
    model.add(Dropout({{uniform(0, 1)}}))
    
    conv_second = {{choice(['yes', 'no'])}}
    if conv_second == 'yes':
        model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
        model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
        model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2),strides=2))
        model.add(Dropout({{uniform(0, 1)}}))
    
    model.add(Flatten())
    model.add(Dense({{choice([32, 64, 128, 256])}}, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout({{uniform(0, 1)}}))
    
    dense_second = {{choice(['yes', 'no'])}}
    if dense_second == 'yes':
        model.add(Dense({{choice([32, 64, 128, 256])}}, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout({{uniform(0, 1)}}))
    
    model.add(Dense(10, activation='softmax'))
        
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

    model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size,
        verbose=1,
        workers=1,#f you want multiprocessing change it
        use_multiprocessing=False,)
    
    score, acc = model.evaluate_generator(generator=test_generator, steps=nb_test_samples, verbose = 1)
    print('Test accuracy:', acc)
    
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

train_generator, validation_generator, test_generator = data()
best_run, best_model = optim.minimize(model=model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=30,
                                      verbose=1,
                                      trials=Trials(),
                                      notebook_name='fonts_classifier')

print("Evalutation of best performing model:")
print(best_model.evaluate_generator(generator=test_generator, steps=200))
print("Best performing model chosen hyper-parameters:")
print(best_run) 

In [ ]:
best_model.save('best_model_top')
best_model.save_weights('best_model_top_weights')

In [ ]:
best_model_adam = keras.models.load_model('best_model_top')

### Evaluate test score of best_model_adam

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255.,)

test_generator = test_datagen.flow_from_directory(
    directory='./Synthetic_dataset/test/',
    batch_size=32,   
    color_mode='grayscale',
    target_size=(32,32))
acc_adam = []
for i in range(5):
    acc_adam.append(best_model_adam.evaluate_generator(generator=test_generator, steps=200, verbose = 1)[1])
print(np.mean(acc_adam))

### Training and validation visualization

In [ ]:
# Plot training & validation accuracy values
plt.figure(figsize=(10,4))
plt.plot(best_model.history.history['acc'])
plt.plot(best_model.history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.figure(figsize=(10,4))
plt.plot(best_model.history.history['loss'])
plt.plot(best_model.history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

### There is hypothesis that with SGD optimizer we can get better score, but accuracy is so close to 1 and it will be very difficult

Lets check it!

There is duplicated code, it is necessary for hyperas run correctly

In [ ]:
def data():
    img_size = (32, 32)
    train_datagen = ImageDataGenerator(rescale = 1./255.)
    test_datagen = ImageDataGenerator(rescale = 1./255.,)

    train_generator = train_datagen.flow_from_directory(
        directory='./Synthetic_dataset/train',
        batch_size=32,   
        color_mode='grayscale',
        target_size=img_size)
    
    validation_generator = test_datagen.flow_from_directory(
        directory='./Synthetic_dataset/val',
        batch_size=32,
        color_mode='grayscale',
        target_size=img_size)
    
    test_generator = test_datagen.flow_from_directory(
        directory='./Synthetic_dataset/test',
        batch_size=32,
        color_mode='grayscale',
        target_size=img_size)
    
    return train_generator, validation_generator, test_generator

def model(train_generator, validation_generator, test_generator):
    
    nb_train_samples = 1000
    nb_validation_samples = 200
    nb_test_samples = 200
    epochs = 100
    batch_size = 32
    
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 1)))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 1)))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2),strides=2))
    model.add(Dropout({{uniform(0, 1)}}))
    
    conv_second = {{choice(['yes', 'no'])}}
    if conv_second == 'yes':
        model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
        model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
        model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2),strides=2))
        model.add(Dropout({{uniform(0, 1)}}))
    
    model.add(Flatten())
    model.add(Dense({{choice([32, 64, 128, 256])}}, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout({{uniform(0, 1)}}))
    
    dense_second = {{choice(['yes', 'no'])}}
    if dense_second == 'yes':
        model.add(Dense({{choice([32, 64, 128, 256])}}, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout({{uniform(0, 1)}}))
    
    model.add(Dense(10, activation='softmax'))
        
    nesterov = {{choice([True, False])}}
    print('nesterov is', nesterov)
    sgd = SGD(lr = {{uniform(0, 0.3)}}, momentum = {{uniform(0, 1)}}, nesterov=nesterov)
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=sgd)

    model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size,
        verbose=1,
        workers=1,#f you want multiprocessing change it
        use_multiprocessing=False,)
    
    score, acc = model.evaluate_generator(generator=test_generator, steps=nb_test_samples, verbose = 1)
    print('Test accuracy:', acc)
    
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

train_generator, validation_generator, test_generator = data()
best_run, best_model = optim.minimize(model=model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=30,
                                      verbose=1,
                                      trials=Trials(),
                                      notebook_name='fonts_classifier')

print("Evalutation of best performing model:")
print(best_model.evaluate_generator(generator=test_generator, steps=200))
print("Best performing model chosen hyper-parameters:")
print(best_run) 

In [ ]:
best_model.save_weights('best_model_top_sgd_weights')
best_model.save('best_model_top_sgd')

In [ ]:
best_model_sgd = keras.models.load_model('best_model_top_sgd')

### Evaluate test score of best_model_sgd

In [ ]:
acc = []
for i in range(5):
    acc.append(best_model_sgd.evaluate_generator(generator=test_generator, steps=200, verbose = 1)[1])
print(np.mean(acc))

### Training and validation visualization

In [ ]:
# Plot training & validation accuracy values
plt.figure(figsize=(10,4))
plt.plot(best_model.history.history['acc'])
plt.plot(best_model.history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.figure(figsize=(10,4))
plt.plot(best_model.history.history['loss'])
plt.plot(best_model.history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


Lets try train already trained model

In [ ]:
history, score = datagen(best_model_sgd,'./Synthetic_dataset/train', './Synthetic_dataset/val',\
                         './Synthetic_dataset/test', verbose=1, img_size=(32,32), batch_size=32,\
                         epochs=100, nb_train_samples=1000, nb_validation_samples=200, nb_test_samples = 200)

### Evaluate test score of best_model_sgd with one more training

In [ ]:
acc_sgd = []
for i in range(5):
    acc_sgd.append(best_model_sgd.evaluate_generator(generator=test_generator, steps=200, verbose = 1)[1])
print(np.mean(acc_sgd))

In [ ]:
best_model_sgd.save_weights('best_model_top_sgd_weights_second')
best_model_sgd.save('best_model_top_sgd_second')

### Training and validation visualization

In [ ]:
# Plot training & validation accuracy values
plt.figure(figsize=(10,4))
plt.plot(best_model_sgd.history.history['acc'])
plt.plot(best_model_sgd.history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.figure(figsize=(10,4))
plt.plot(best_model_sgd.history.history['loss'])
plt.plot(best_model_sgd.history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

## Conclsuion

We trained 2 models with identity architecture:

    INPUT - [CONV - CONV - CONV - POOL]*2 - [DENSE]*2 - OUTPUT
    
One has trained with Adam optimizator, another with SGD.

### Results:
    
With Adam optimizer:

In [ ]:
print('Test accuracy is', np.mean(acc_adam))

With SGD optimizer:

In [ ]:
print('Test accuracy is', np.mean(acc_sgd))